Google Colab was used to perfom keras tuning, which enables GPU acceleration and hence perfoms the tuning significantly faster than perfmoing it locally on a computer without a GPU

In [2]:
pip install -U keras-tuner

     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=28a4e0bd11cb93ba5506a89d51552dc63ff904ae4618b32f990216667660a8ce
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=5b2d7a578041ab80b4934e55f46aefb30db02addeb4b5533dd5b75f111931988
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
from google.colab import files
uploaded = files.upload()

Saving X_test.npy to X_test.npy
Saving X_train.npy to X_train.npy
Saving y_test.npy to y_test.npy
Saving y_train.npy to y_train.npy


In [4]:
!ls

sample_data  X_test.npy  X_train.npy  y_test.npy  y_train.npy


Import the train and test .npy files 


In [47]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from kerastuner.tuners import RandomSearch
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

EPOCHS = 10


X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

X_train = X_train/255
X_test = X_test/255



Create ImageDataGenerator to effectively increase the size of the training dataset

In [58]:
train_datagenerator = ImageDataGenerator(shear_range  = 0.2,
                                   zoom_range         = 0.2,
                                   horizontal_flip    = True,
                                   rotation_range     = 40,
                                   width_shift_range  = 0.2,
                                   height_shift_range = 0.2,
                                   validation_split = 0.1)

trainset_gen = train_datagenerator.flow(X_train, y_train, subset="training")
validation_gen = train_datagenerator.flow(X_train, y_train, subset="validation")

In [ ]:
!ls

sample_data  X_test.npy  X_train.npy  y_test.npy  y_train.npy


In [75]:
!rm -rf Tuner_models


##Perform Tuning


In [76]:

def build_model(hp):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1), padding='same'))
    

    for i in range(hp.Int('num_convLayers', 2, 5)):
        model.add(Conv2D(hp.Choice(f"filters_{i}", values = [32,64]), (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        


    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))


    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials = 100,
    project_name="Tuner_models"
)

tuner.search(trainset_gen,epochs=10,validation_data = validation_gen, callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=2)])


Trial 98 Complete [00h 00m 31s]
val_accuracy: 0.8891013264656067

Best val_accuracy So Far: 0.9196940660476685
Total elapsed time: 00h 48m 55s
INFO:tensorflow:Oracle triggered exit


In [ ]:
!ls

sample_data  tuner_models  X_test.npy  X_train.npy  y_test.npy	y_train.npy


In [80]:
best_model = tuner.get_best_models()[0]
best_model.summary()
test_accuracy = best_model.evaluate(X_test,y_test)
print(f'Testing Accuracy: {(test_accuracy[1] * 100)}%')
best_model.save("./best_model.h5")


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0

$\boxed{\text{Testing accuracy achieved with the unseen images is 89.7%}}$

In [81]:
!ls

best_model.h5  Tuner_models  X_train.npy  y_train.npy
sample_data    X_test.npy    y_test.npy


In [82]:
files.download('best_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r tuners.zip ./Tuner_models

In [84]:
files.download('tuners.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>